<a href="https://colab.research.google.com/github/Tamilselvan0101/HR_AI_assistant_analyzes_your_leave_details_and_company_policy_and_gives_suggestions_approval/blob/main/HR_AI_assistant_analyzes_your_leave_details_and_company_policy_and_gives_suggestions_approval_and_formalities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-bigquery
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path1= ["/content/drive/MyDrive/github_projects/fake_face_book hr policy.pdf"]
path2= "/content/drive/MyDrive/github_projects/pizza-412109-78beb3072fb9.json"

## **Enter your api key 👇**

In [ ]:
#pls enter your API key HERE
APIKEY=

In [ ]:
#from re import template
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

employee_id=input("enter employee id: ")
querry= input(" enter your query : ")


#extratct pdf text
conts=path1
paragraph =" "
for path in conts:
  pdreader = PdfReader(path)
  for i, page in enumerate(pdreader.pages):
    content = page.extract_text()
    if content:
        paragraph += content

text_splitter = CharacterTextSplitter(separator = "\n",chunk_size = 1200,chunk_overlap  = 200,length_function = len)
texts = text_splitter.split_text(paragraph)

embeddings = OpenAIEmbeddings(openai_api_key=APIKEY)
storage = FAISS.from_texts(texts, embeddings)

from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
chain = load_qa_chain(OpenAI(openai_api_key=APIKEY), chain_type="stuff")


from google.cloud import bigquery
from google.oauth2 import service_account

#key_path= "/content/hrproject-412815-6f13d1aeb8b5.json"
#project_id="hrproject-412815"

key_path = path2
project_id = "pizza-412109"


#bigquery
def client_query(key_path,project_id,query):
    credentials = service_account.Credentials.from_service_account_file(
        key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
        )

    client = bigquery.Client(credentials=credentials, project=project_id)
    query_job = client.query(query)  # Make an API request.
    data=query_job.to_dataframe()
    #print("The query data:")
    return(data)
    """
    for row in query_job:
        # Row values can be accessed by field name or index.
        print("name={}, count={}".format(row[0], row["total_people"])) """


fetch_query='''(SELECT * FROM `pizza-412109.pizza123.emp_rem_leave` where employee_id= '{}')'''.format(employee_id)

fetch= client_query(key_path,project_id,query=fetch_query)

list_of_lists = fetch.values.tolist()

print(list_of_lists)
nam= list_of_lists [0][2]
mail= list_of_lists[0][1]
emplid= list_of_lists[0][0]
pl_cl= list_of_lists[0][3]
sl= list_of_lists[0][4]
maternity= list_of_lists[0][5]
comp_off= list_of_lists[0][6]
carry_forward= list_of_lists[0][7]
lop= list_of_lists[0][8]
wfh=list_of_lists[0][9]

#print("value is ")
#print(pl_cl)


template= ("i am a employe my name is {} my employee id {} my mail id is {} i have {} pl_cl remaining , {} sick leave remaining , {} maternity leave remaining , {} cam_off remaining , {} leaves carry_forward,{} lop remaining  ,{} wfh  remaining").format( nam,mail,emplid,pl_cl,sl,maternity,comp_off,carry_forward,lop, wfh )
#print(template)


#simalirity search
def search1(query):
    docs = storage.similarity_search(query)
    tas=chain.run(input_documents=docs, question=query)
    #result_label.config(text=f"{tas}")
    return(tas)
search_reasult=search1(query=querry)

#print(search_reasult)


#finetuning
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(openai_api_key=APIKEY)
from langchain.prompts import (ChatPromptTemplate,HumanMessagePromptTemplate,
MessagesPlaceholder,SystemMessagePromptTemplate)
tass=""

infom=("You are a nice hr  you will give suggestions and steps by referring the company policy, the compny policy is {} and this information about my self {}  ").format(search_reasult,template)

prompt = ChatPromptTemplate(messages=[
    SystemMessagePromptTemplate.from_template(infom),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{question}")])
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(llm=llm, prompt=prompt, verbose=True, memory=memory)

def search():
    query =querry
    #docs = storage.similarity_search(query)
    tapp = conversation({"question": query})
    """docs[0].page_content += tapp['text']
    tass = chain.run(input_documents=docs, question=query)
    print(tapp)
    """
    #print(tapp)
    tass= tapp.get('text')
    print(tass)


search()

# pls use X/CH/866 for employee id or check the bigquery


enter employee id: X/CH/866
 enter your query : am i eligeble for maternity leave


[['X/CH/866', 'gangadhar', 'gangadhar@omail.com', 3, 12, 0, 2, 0, 0, 0]]
i am a employe my name is gangadhar@omail.com my employee id gangadhar my mail id is X/CH/866 i have 3 pl_cl remaining , 12 sick leave remaining , 0 maternity leave remaining , 2 cam_off remaining , 0 leaves carry_forward,0 lop remaining  ,0 wfh  remaining
 Based on the information provided, it appears that you may be eligible for maternity leave if you are a female employee with less than two surviving children and have completed at least 80 days of working days from the date of joining. However, it is best to confirm with your HR Manager and supervisor and provide them with the necessary documentation, such as an approved Medical Practitioner's report confirming your pregnancy and estimated delivery date, to officially request and be approved for maternity leave.


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice hr  you will give suggestions and steps by referring the company pol